In [ ]:
#import data automatically
# Source:
# Bätzing J, Holstiege J, Hering R, Akmatov MK, Steffen A, Dammertz L, Czihal T, von Stillfried D.
# Häufigkeiten von Vorerkrankungen mit erhöhtem Risiko für einen schwerwiegenden klinischen Verlauf von COVID-19 – Eine Analyse kleinräumiger Risikoprofile in der deutschen Bevölkerung.
# Zentralinstitut für die kassenärztliche Versorgung in Deutschland (Zi).
# Versorgungsatlas-Bericht Nr. 20/05.
#Berlin 2020
# DOI: 10.20364/VA-20.05
# Link: https://www.versorgungsatlas.de/themen/alle-analysen-nach-datum-sortiert/?tab=2&uid=110
#REQUIREMENTS:
# Selenium package: pip install selenium
# selenium-3.141.0 installed
# Firefow Browser: 
# https://www.mozilla.org/de/firefox/new/
# PATH :
#WebDriverException: Message: 'geckodriver' executable needs to be in PATH. 
#Download geckodriver: https://github.com/mozilla/geckodriver/releases
# Save it in directory


from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.chrome.options import Options
from datetime import datetime
import shutil # for deleting non-empties directories
import os # file tree navigation
import re # regex expressions
from pathlib import Path # Requirement:Python >=3.5

In [ ]:
# We create the Downloads directory in working directory
# For the uniqueness of the directory name we use simple method:the timestamp trick

# Do you want to erase all *_Downloads directories in working directory?
erase_directories=input("Do you want to delete all subdirectories with suffix '_Downloads?Y/N")
erase_directories=erase_directories.upper()
if erase_directories == 'Y':
    erase_directories=True
else:
    erase_directories=False

# Delete all contents of a directory using shutil.rmtree() and  handle exceptions

for root,dirs,files in os.walk('.'):
    if re.search(r'_Downloads$', root) and erase_directories:
            try:
              shutil.rmtree(root)
            except:
               print('Error while deleting directory')
# Create the directory
created=False

while(not(created)):
    ts = datetime.now().strftime("%Y%m%d%H%M%S")
    new_dir_name=ts + "_Downloads"
    new_dir_path=os.path.join(os.getcwd(),new_dir_name)
    Path(new_dir_path).mkdir(parents=True, exist_ok=True)
    created=os.path.exists(new_dir_path)
download_dir=new_dir_path + "\\"
print("Files will be downloaded to: "+download_dir)

In [ ]:

options = Options()

options.add_experimental_option("prefs", {
  "download.default_directory": download_dir,
  "download.prompt_for_download": False,
  "download.directory_upgrade": True,
  "safebrowsing.enabled": True
})
#open the webpage with Chrome
browser=webdriver.Chrome(options=options)
browser.get("https://www.versorgungsatlas.de/themen/alle-analysen-nach-datum-sortiert/?tab=2&uid=110")

#select the Tab "Tabellen"
Tables_Tab = "//span[contains(text(),'Tabellen')]"
selected_Tab = browser.find_element_by_xpath(Tables_Tab)
selected_Tab.click()

# Select Filter 1:We want to download only for Region =KV
filter_1 = Select(browser.find_element_by_id('region'))
filter_1.select_by_visible_text("KV")

#Select filter 2:  "Risikopatienten": We download a file pro an option of this filter
filter_2_xpath="//select[@id='auspraegung']"

#we get firstly the list of all options in the filter 2
filter_2 = Select(browser.find_element_by_id('auspraegung'))
options_list=[]
for option in filter_2.options:
   op=option.text
   options_list.append(op)
#Then we go through the option list and silently download a file pro option list
for op in options_list:
    filter_2 = Select(browser.find_element_by_id('auspraegung'))
    filter_2.select_by_visible_text(op)
    file_link= browser.find_element_by_xpath("//u[contains(text(),'Daten als Excel-Datei exportieren')]")
    file_link.click()

browser.quit()
